# Client Data Only Model clientmodel3a

- feature selection on PenFed data + Origence
- without LTV application feature
- train test split: (with april 2021- june 2021 removed)
- 
  "train": {"start_date": "2020-07-01", "end_date": "2022-07-01"},
  "valid": {"start_date": "2022-07-01", "end_date": "2023-01-01"},
  "test": {"start_date": "2023-01-01", "end_date": "2024-01-01"}

In [23]:
# House keeping settings
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False
import numba
n_jobs = os.getenv('CPU_LIMIT', default=-1)
n_jobs = int(float(n_jobs)/2)
numba.set_num_threads(n_jobs)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import os
import pandas as pd
import json
import numpy as np

from model_engine.assets.utils import load_asset
from model_engine.io.loaders import load_json
from model_engine.power.post_sale import PowerModelBuilder
from model_engine.analysis.suggesters import evaluate_datesplits
import warnings

from zaml.common.utils import load_state

import model_engine, zaml
model_engine.__version__, zaml.__version__

('v1.13.1', '34.4.1')

In [25]:
from model_engine.model_builder.asset_parser import asset_parser
from model_engine.model_builder import ModelBuilder

In [26]:
input_model_id = 'clientmodel4a'
input_target_name = 'final_DQ90_m12'


final_model_output_path = f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{input_model_id}'


print(f'Final model path: {final_model_output_path}')

Final model path: /d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel4a


In [27]:
asset = json.load(open(f'./{input_model_id}_asset.json'))

In [28]:
asset['config']['data_split'] 

{'train': {'start_date': '2020-07-01', 'end_date': '2022-07-01'},
 'valid': {'start_date': '2022-07-01', 'end_date': '2023-01-01'},
 'test': {'start_date': '2023-01-01', 'end_date': '2024-01-01'}}

In [29]:
national_data_asset = json.load(open('/d/shared/users/lyt/penfed_custom_model_national_models/nationalmodel3a/asset.json'))
print(len(national_data_asset['config']['exclusion_list']))

asset['config']['exclusion_list']= national_data_asset['config']['exclusion_list']

3328


In [30]:
# model_builder = ModelBuilder(asset)
# model_builder.configure_run(final_model_output_path, 
#                             cache=True) 

# state = asset_parser(asset)

# input_asset = (json.dumps(state['input_asset'], indent=4))
# print(input_asset)

In [31]:
# with open (os.path.join(final_model_output_path, 'input_configuration.json'), 'w') as outfile:
#     outfile.write(input_asset)

In [32]:
# from model_engine.model_builder import build_model

# output = build_model.build_model(asset, final_model_output_path)

In [33]:
scoreFiles = []

for j in ['_auc.json','_ks.json']:
#     for i in ['train', 'valid', 'test']:
    for i in ['train','test',]:
        scoreFiles.append(i+j)
        
scoreFiles_model = {}
for i in scoreFiles:
    name = i.split('.')[0]
    # print(name)
    scoreFiles_model[name] = json.load(open(os.path.join(final_model_output_path, i), 'rb'))


for i in scoreFiles_model:
    print(i, scoreFiles_model[i])

train_auc {'auc': 0.7998192905183084}
test_auc {'auc': 0.7582703744338473}
train_ks {'ks': 0.45219997933849176}
test_ks {'ks': 0.38411355562782196}


In [34]:
features_importance = pd.read_parquet(os.path.join(final_model_output_path, 'feature_importance.parquet'))
features_importance.shape

(248, 2)

In [35]:
features_importance.head(20)

,feature,importance
0,inq_inquiries__count_by_indCode_auto_mortgage,0.003416
1,trade_blnc_to_hc__max_by_acctType_auto,0.002604
2,inq_inquiries_in_last_3_months__sum,0.001959
3,trade_clAmt__mean_by_prtfType_revolving,0.001936
4,trade_clAmt_accts_with_rcntPmt__mean_by_prtfTy...,0.001830
5,trade_clAmt_all_non_derog_accts__mean_by_prtfT...,0.001788
6,inq_inquiries_in_last_9_months__sum,0.001773
7,trade_clAmt_active_accts__mean_by_prtfType_rev...,0.001705
8,trade_clAmt_accts_with_rcntPmt__sum_by_prtfTyp...,0.001639
9,trade_clAmt_accts_with_rcntPmt__max_by_prtfTyp...,0.001580


In [36]:
final_model_output_path

'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel4a'

In [37]:
!ls /d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel4a

artifact_manifest.json			test_scores.parquet
asset.json				test_target.parquet
best_model_params.json			test_zest_scores.parquet
calibration_object.obj			top_features.parquet
feature_definition.parquet		train_app.parquet
feature_importance.parquet		train_auc.json
fit_time_info.json			train_data_summary.json
input_configuration.json		train_fe_data.parquet
keep_features.json			train_history.json
key_factors_mapping.json		train_ks.json
model.obj				train_scores.parquet
model_strategy.json			train_target.parquet
mrm_pipeline.obj			train_zest_scores.parquet
parsed_monotonic_constraints_list.json	valid_app.parquet
pipeline.obj				valid_auc.json
score_recalibration_mapping.json	valid_data_summary.json
splitter.obj				valid_fe_data.parquet
static_asset.json			valid_ks.json
test_app.parquet			valid_scores.parquet
test_auc.json				valid_target.parquet
test_data_summary.json			valid_zest_scores.parquet
test_fe_data.parquet			value_based_key_factor_mapping.json
test_ks.json				versions.json


# Unfold Model 

In [38]:
unfold_model_id = 'clientmodel4b'


unfold_model_output_path = f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{unfold_model_id}'


print(f'Unfold model path: {unfold_model_output_path}')

Unfold model path: /d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel4b


In [39]:
# Feature selection model 

base_model_id = 'clientmodel4a'
base_asset = json.load(open(f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{base_model_id}/asset.json'))
keep_features = json.load(open(f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{base_model_id}/keep_features.json'))
base_asset['config']

{'data_split': {'train': {'start_date': '2020-07-01',
   'end_date': '2022-07-01'},
  'valid': {'start_date': '2022-07-01', 'end_date': '2023-01-01'},
  'test': {'start_date': '2023-01-01', 'end_date': '2024-01-01'}},
 'target': 'final_DQ90_m12',
 'memory_efficient': True,
 'pipeline_factory': {'name': 'AutoValidPipelineFactory',
  'transformers': [{'zaml_class': 'LevelSelection',
    'params': {'thresh': 0.01, 'change_to': 'Other', 'encoding': 'onehot'}},
   {'zaml_class': 'FillNA', 'params': {'replace_by': -1, 'add_flags': False}},
   {'zaml_class': 'CorrelationFeatureSelection',
    'params': {'corr_threshold': 0.995},
    'fit_params': {'ignore_columns': ['trade_months_since_openDate__max',
      'trade_acctType__count',
      'trade_number_DQ30_24_months__sum',
      'trade_number_DQ60_24_months__sum',
      'trade_number_DQ90_24_months__sum',
      'trade_blncAmt_all_derog_accts__max',
      'trade_blncAmt_active_accts__sum',
      'trade_blncAmt_active_accts__sum_by_prtfType_rev

In [40]:
# Default hyperparameters for large client pipelines

base_asset['config']['pipeline_factory'] = "LargeClientPipelineFactory"

In [41]:
base_asset['config'].pop('mandatory_features')
base_asset['config']['base_features'] = f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{base_model_id}/keep_features.json'

In [42]:
base_asset['data']['client_data']

{'data': '#/home/psk/client-project-penfed/autoindirect/modeling/origencepoc_model/client_models/sample_files/client_data_wo_ltv_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/home/psk/client-project-penfed/autoindirect/playground/psk/keep_keys.json'},
 'asset': {'feature_engineering': 'one_to_one',
  'validator': 'base',
  'table_type': 'app',
  'info': {'key': 'ZEST_KEY'}}}

In [43]:
for t, l in keep_features.items():
    if t != 'client_data': 
        base_asset['data'][t]['io_params']['keep_features'] = l

In [44]:
# model_builder = ModelBuilder(base_asset)
# model_builder.configure_run(unfold_model_output_path, 
#                             cache=True) 

# state = asset_parser(base_asset)

# input_asset = (json.dumps(state['input_asset'], indent=4))
# print(input_asset)

In [45]:
# with open (os.path.join(unfold_model_output_path, 'input_configuration.json'), 'w') as outfile:
#     outfile.write(input_asset)

In [46]:
from model_engine.model_builder import build_model

output = build_model.build_model(base_asset, unfold_model_output_path)

parsing model-builder asset


Configuring model builder


INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_asset>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_data>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <monotonic_constraints_list>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <data_split>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <train_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <train_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <valid_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <valid_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <test_sample_weight>...
INFO:zaml.artifa

building model


INFO:zaml.artifact_engine.logger:Finished <parsed_monotonic_constraints_list>, total time spent: 0:00:00.677899
INFO:zaml.artifact_engine.logger:Executing SplitterArtifact <splitter>...
INFO:zaml.artifact_engine.logger:Finished <splitter>, total time spent: 0:00:00.000826
INFO:zaml.artifact_engine.logger:Executing SeriesArtifact <target>...
INFO:zaml.artifact_engine.logger:Finished <target>, total time spent: 0:00:00.005443
INFO:zaml.artifact_engine.logger:Executing AppArtifact <app>...
INFO:zaml.artifact_engine.logger:Finished <app>, total time spent: 0:00:00.009781
INFO:zaml.artifact_engine.logger:Executing DataArtifact <data>...
INFO:zaml.artifact_engine.logger:Finished <data>, total time spent: 0:00:00.000543
INFO:zaml.artifact_engine.logger:Executing PipeFactoryConfigArtifact <pipe_factory_config>...
INFO:zaml.artifact_engine.logger:Finished <pipe_factory_config>, total time spent: 0:00:00.095880
INFO:zaml.artifact_engine.logger:Executing ExclusionListParser <parsed_exclusion_list

...

-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 248
Time spent: 5.957s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 248
Time spent: 0.001s


INFO:zaml.artifact_engine.logger:Executing FeDataArtifact <test_fe_data>...


-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: bnkr
Transformer type: None
Number of features: 8
Time spent: 0.000s
-------------------------
Name: client_data
Transformer type: None
Number of features: 537
Time spent: 0.000s
-------------------------
Name: collec
Transformer type: None
Number of features: 5
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 13
Time spent: 0.000s
-------------------------
Name: member
Transformer type: None
Number of features: 28
Time spent: 0.000s
-------------------------
Name: trade
Transformer type: None
Number of features: 194
Time spent: 0.000s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.079s
-------------------------
Name: bnkr FE
Transformer type: ManyToOneEngine
Number of features: 8
Time spent: 0.019s
-------------------------
Name: client_data FE
T

-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 248
Time spent: 5.139s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 248
Time spent: 0.001s


INFO:zaml.artifact_engine.logger:Finished <test_fe_data>, total time spent: 0:00:10.982968
INFO:zaml.artifact_engine.logger:Executing StaticAssetArtifact <static_asset>...
INFO:zaml.artifact_engine.logger:Finished <static_asset>, total time spent: 0:00:06.513511
INFO:zaml.artifact_engine.logger:Executing TrainHistoryArtifact <train_history>...
INFO:zaml.artifact_engine.logger:Finished <train_history>, total time spent: 0:00:00.001017
INFO:zaml.artifact_engine.logger:Executing BestModelParamsArtifact <best_model_params>...
INFO:zaml.artifact_engine.logger:Finished <best_model_params>, total time spent: 0:00:00.000383
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <train_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_scores>, total time spent: 0:00:02.626968
INFO:zaml.artifact_engine.logger:Executing CalibrationObjectArtifact <calibration_object>...
INFO:zaml.artifact_engine.logger:Finished <calibration_object>, total time spent: 0:00:06.840753
INFO:zaml.artifact_

In [47]:
scoreFiles = []

for j in ['_auc.json','_ks.json']:
#     for i in ['train', 'valid', 'test']:
    for i in ['train','test',]:
        scoreFiles.append(i+j)
        
scoreFiles_model = {}
for i in scoreFiles:
    name = i.split('.')[0]
    # print(name)
    scoreFiles_model[name] = json.load(open(os.path.join(unfold_model_output_path, i), 'rb'))


for i in scoreFiles_model:
    print(i, scoreFiles_model[i])

train_auc {'auc': 0.795817091194695}
test_auc {'auc': 0.757834116027595}
train_ks {'ks': 0.4461054389131328}
test_ks {'ks': 0.3867804202901779}


# Fold Valid Model 

In [48]:
fold_valid_model_id = 'clientmodel4c'
fold_valid_model_output_path = f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{fold_valid_model_id}'


print(f'fold valid model path: {fold_valid_model_output_path}')

fold valid model path: /d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel4c


In [49]:
unfold_model_id = 'clientmodel4b'
asset = json.load(open(f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{unfold_model_id}/asset.json'))
keep_features = json.load(open(f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{unfold_model_id}/keep_features.json'))
best_ntree = json.load(open(f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{unfold_model_id}/best_model_params.json'))
best_ntree['best_ntree_limit']

2914

In [50]:
asset['config']['data_split'] 

{'train': {'start_date': '2020-07-01', 'end_date': '2022-07-01'},
 'valid': {'start_date': '2022-07-01', 'end_date': '2023-01-01'},
 'test': {'start_date': '2023-01-01', 'end_date': '2024-01-01'}}

In [51]:
asset['config']['data_split'] = {
    'train': {'start_date': '2020-07-01', 'end_date': '2023-01-01'},
    'test': {'start_date': '2023-01-01', 'end_date': '2024-01-01'}}

In [52]:
asset['config']['pipeline_factory'] =  {
      "transformers": [
        {
          "zaml_class": "LevelSelection",
          "params": {
            "thresh": 0.01,
            "change_to": "Other",
            "encoding": "onehot"
          }
        },
        {
          "zaml_class": "FillNA",
          "params": {
            "replace_by": -1,
            "add_flags": False
          }
        },
        {
          "zaml_class": "GeneralFeatureSelection",
          "params": {
            "features": [
              "app_is_client"
            ]
          }
        }
      ],
      "model": {
        "zaml_class": "XGBoostModel",
        "params": {
          "n_estimators": best_ntree['best_ntree_limit'],
          "learning_rate": 0.01,
          "max_depth": 3,
          "subsample": 0.5,
          "scale_pos_weight": 2.5,
          "colsample_bytree": 0.05,
          "min_child_weight": 350,
          "seed": 12,
          "eval_metric": "auc"
        }
      }
    }

In [53]:
asset['config']['pipeline_factory']['model']['params']['n_estimators'] 

2914

In [55]:
# asset['config'].pop('mandatory_features')
asset['config']['fold_valid'] = True
asset['config']['enforce_artifact_validation'] = True
asset['config']['base_features'] = f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{base_model_id}/keep_features.json'


In [56]:
for t, l in keep_features.items():
    print(t)
    if t != 'client_data': 
        asset['data'][t]['io_params']['keep_features'] = l
    else: 
        continue

trade
inq
collec
bnkr
member
client_data


In [57]:
asset['data']['client_data']

{'data': '#/home/psk/client-project-penfed/autoindirect/modeling/origencepoc_model/client_models/sample_files/client_data_wo_ltv_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/home/psk/client-project-penfed/autoindirect/playground/psk/keep_keys.json'},
 'asset': {'feature_engineering': 'one_to_one',
  'validator': 'base',
  'table_type': 'app',
  'info': {'key': 'ZEST_KEY'}}}

In [58]:
model_builder = ModelBuilder(asset)
model_builder.configure_run(fold_valid_model_output_path, 
                            cache=True) 

state = asset_parser(asset)

input_asset = (json.dumps(state['input_asset'], indent=4))
print(input_asset)

{
    "data": {
        "trade": {
            "asset": "equifax/cms_6/trade.json",
            "data": "#/home/psk/client-project-penfed/autoindirect/modeling/origencepoc_model/client_models/sample_files/trade_sampled_files.json",
            "io_params": {
                "drop_duplicates": true,
                "keep_keys": "/home/psk/client-project-penfed/autoindirect/playground/psk/keep_keys.json",
                "keep_features": [
                    "trade_acctType__count",
                    "trade_blncAmt__mean_by_acctType_auto",
                    "trade_blncAmt__min_by_acctType_education",
                    "trade_blncAmt_accts_never_dq__max",
                    "trade_blncAmt_accts_never_dq__max_by_ecoa_joint",
                    "trade_blncAmt_accts_never_dq__max_by_prtfType_mortgage",
                    "trade_blncAmt_accts_never_dq__mean_by_acctType_unsecure",
                    "trade_blncAmt_accts_never_dq__mean_by_ecoa_joint",
                    "trade_blncA

In [59]:
with open (os.path.join(fold_valid_model_output_path, 'input_configuration.json'), 'w') as outfile:
    outfile.write(input_asset)

In [60]:
from model_engine.model_builder import build_model

output = build_model.build_model(asset, fold_valid_model_output_path)

parsing model-builder asset


Configuring model builder


INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_asset>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_data>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <monotonic_constraints_list>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <data_split>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <train_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <train_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <test_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <test_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <pipeline_factory>...
INFO:zaml.artifact_e

building model


INFO:zaml.artifact_engine.logger:Finished <parsed_monotonic_constraints_list>, total time spent: 0:00:00.696162
INFO:zaml.artifact_engine.logger:Executing SplitterArtifact <splitter>...
INFO:zaml.artifact_engine.logger:Finished <splitter>, total time spent: 0:00:00.000788
INFO:zaml.artifact_engine.logger:Executing SeriesArtifact <target>...
INFO:zaml.artifact_engine.logger:Finished <target>, total time spent: 0:00:00.004897
INFO:zaml.artifact_engine.logger:Executing AppArtifact <app>...
INFO:zaml.artifact_engine.logger:Finished <app>, total time spent: 0:00:00.009457
INFO:zaml.artifact_engine.logger:Executing DataArtifact <data>...
INFO:zaml.artifact_engine.logger:Finished <data>, total time spent: 0:00:00.000769
INFO:zaml.artifact_engine.logger:Executing PipeFactoryConfigArtifact <pipe_factory_config>...
INFO:zaml.artifact_engine.logger:Finished <pipe_factory_config>, total time spent: 0:00:00.000358
INFO:zaml.artifact_engine.logger:Executing ExclusionListParser <parsed_exclusion_list

...


INFO:zaml.artifact_engine.logger:Finished <train_fe_data>, total time spent: 0:00:27.323813
INFO:zaml.artifact_engine.logger:Executing FeDataArtifact <test_fe_data>...


-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 248
Time spent: 3.537s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 248
Time spent: 0.001s
-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: bnkr
Transformer type: None
Number of features: 8
Time spent: 0.000s
-------------------------
Name: client_data
Transformer type: None
Number of features: 537
Time spent: 0.000s
-------------------------
Name: collec
Transformer type: None
Number of features: 5
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 13
Time spent: 0.000s
-------------------------
Name: member
Transformer type: None
Number of features: 28
Time spent: 0.000s
-------------------------
Name: trade
Transformer type: None
Number of features: 194
Time spent: 0.000s
-------------------------
Na


INFO:zaml.artifact_engine.logger:Finished <test_fe_data>, total time spent: 0:00:04.663413
INFO:zaml.artifact_engine.logger:Executing StaticAssetArtifact <static_asset>...


-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 248
Time spent: 2.136s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 248
Time spent: 0.001s


INFO:zaml.artifact_engine.logger:Finished <static_asset>, total time spent: 0:00:02.240712
INFO:zaml.artifact_engine.logger:Executing TrainHistoryArtifact <train_history>...
INFO:zaml.artifact_engine.logger:Finished <train_history>, total time spent: 0:00:00.000595
INFO:zaml.artifact_engine.logger:Executing BestModelParamsArtifact <best_model_params>...
INFO:zaml.artifact_engine.logger:Finished <best_model_params>, total time spent: 0:00:00.000366
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <train_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_scores>, total time spent: 0:00:01.066180
INFO:zaml.artifact_engine.logger:Executing CalibrationObjectArtifact <calibration_object>...
INFO:zaml.artifact_engine.logger:Finished <calibration_object>, total time spent: 0:00:03.248229
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <train_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_submodel_scores>, total time spent: 0:00:00.00061

In [62]:
scoreFiles = []

for j in ['_auc.json','_ks.json']:
#     for i in ['train', 'valid', 'test']:
    for i in ['train','test',]:
        scoreFiles.append(i+j)
        
scoreFiles_model = {}
for i in scoreFiles:
    name = i.split('.')[0]
    # print(name)
    scoreFiles_model[name] = json.load(open(os.path.join(fold_valid_model_output_path, i), 'rb'))


for i in scoreFiles_model:
    print(i, scoreFiles_model[i])

train_auc {'auc': 0.7904910903929925}
test_auc {'auc': 0.7592454162168837}
train_ks {'ks': 0.43267032821692997}
test_ks {'ks': 0.3940642510549115}
